# MLP

In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

# Data loader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Model

In [4]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.relu(out)
        out = self.mlp2(out)
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)

* Adam Optimizer
* lr : learning rate

In [5]:
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 최적화 알고리즘 class 선언

* 순전파(forward propagation) :뉴럴 네트워크 모델의 입력층부터 출력층까지 순서대로 변수들을 계산하고 저장하는 것을 의미
  * 딥러닝이 학습을 하면서 자신만의 답을 출력하는 과정이다. 모델에서 정답을 뽑아내는 과정
* 역전파(back propagation) : 순전파 (Feedforward) 알고리즘 에서 발생한 오차를 줄이기 위해 새로운 가중치를 업데이트하고, 새로운 가중치로 다시 학습하는 과정
  * 중간 변수와 파라미터에 대한 그래디언트(gradient)를 반대 방향으로 계산하고 저장
  * loss가 작아지는 최적의 경로는 찾을 수 없지만, 지금 현재 시점에서 loss를 작게 만드는 최적의 방향 => 그레디언트의 반대 방향으로
  * 그레디언트(gradient) : 벡터가 error space에서 가장 급격하게 에러가 증가하는 방향성

In [6]:
# 뉴럴 네트워크 모델 학습
total_epochs = 3
print('number of iteration :', len(train_loader)) # 몇 개의 iter로 되어있는가
# epoch : 모든 데이터를 한 번 학습하는 단위
for epoch in range(total_epochs):
    # iteration : 한 'mini-batch' 단위의 데이터를 학습하는 단위
    for i, (images, labels) in enumerate(train_loader):  
        # images : [mini-batch, 1, 28, 28]
        # labels : [mini-batch]
        images = images.reshape(-1, 28*28).to(device) 
        labels = labels.to(device)
        
        # Forward pass : input을 넣을 때 ouput을 내는 weight 구하는 과정
        outputs = model(images)
        ce_loss = CELoss(outputs, labels)
        
        # Backward and optimize
        adam_optimizer.zero_grad() # 다양한 optimization 기법 적용 가능 => gradient 초기화
        ce_loss.backward() # Back propagation
        adam_optimizer.step() # optimizer 작동
            
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, ce_loss.item()))

number of iteration : 469
Epoch [1/3], Loss: 0.3107
Epoch [2/3], Loss: 0.2814
Epoch [3/3], Loss: 0.1423


* 모델 성능 테스트 => back propgation 과정을 수행하지 않음

In [7]:
# 학습이 끝난 후 모델 성능 테스트
# test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해
with torch.no_grad(): # gradient 계산하지 않도록 하는 코드
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # 모델이 뽑은 output 중에 최대값
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # 예측을 잘 한 것

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 95.82 %


In [9]:
correct

9582

In [10]:
# 학습한 모델을 model.ckpt라는 이름으로 저장
torch.save(model.state_dict(), 'model_basic.ckpt')

# 최적화 함수

* Stochastic Gradient Descent, momentum, Adagrad, RMSprop, Adam

In [11]:
# Stochastic Gradient Descent
sgd_optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Stochastic Gradient Descent with momentum
sgd_with_momentum_optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Adagrad
Adagrad_optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

# RMSprop
RMSprop_optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)

# Adam
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# 뉴럴 네트워크 모델 학습
total_epochs = 3
for epoch in range(len(train_loader)):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        ce_loss = CELoss(outputs, labels)
        
        # Backward and optimize
        sgd_optimizer.zero_grad()
        ce_loss.backward() # Back propagation
        sgd_optimizer.step() # optimizer 작동
            
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, ce_loss.item()))

Epoch [1/3], Loss: 0.1241
Epoch [2/3], Loss: 0.1034
Epoch [3/3], Loss: 0.2531
Epoch [4/3], Loss: 0.1083
Epoch [5/3], Loss: 0.1863
Epoch [6/3], Loss: 0.1656
Epoch [7/3], Loss: 0.1415
Epoch [8/3], Loss: 0.1886
Epoch [9/3], Loss: 0.1162
Epoch [10/3], Loss: 0.0695
Epoch [11/3], Loss: 0.0575
Epoch [12/3], Loss: 0.1460
Epoch [13/3], Loss: 0.0901
Epoch [14/3], Loss: 0.1300
Epoch [15/3], Loss: 0.0607
Epoch [16/3], Loss: 0.1378
Epoch [17/3], Loss: 0.0253
Epoch [18/3], Loss: 0.0945
Epoch [19/3], Loss: 0.1846
Epoch [20/3], Loss: 0.0713
Epoch [21/3], Loss: 0.0637
Epoch [22/3], Loss: 0.0822
Epoch [23/3], Loss: 0.0889
Epoch [24/3], Loss: 0.0854
Epoch [25/3], Loss: 0.0508
Epoch [26/3], Loss: 0.0245
Epoch [27/3], Loss: 0.0597
Epoch [28/3], Loss: 0.1098
Epoch [29/3], Loss: 0.0475
Epoch [30/3], Loss: 0.1687
Epoch [31/3], Loss: 0.0411
Epoch [32/3], Loss: 0.0670
Epoch [33/3], Loss: 0.0514
Epoch [34/3], Loss: 0.1364
Epoch [35/3], Loss: 0.1127
Epoch [36/3], Loss: 0.0813
Epoch [37/3], Loss: 0.1229
Epoch [38/

KeyboardInterrupt: 

# Dropout

* 정규화를 위해 사용 -=> 서로 연결된 연결망(layer)에서 0부터 1 사이의 확률로 뉴런을 제거(drop)하는 기법
  * 어떤 특정한 설명변수 Feature만을 과도하게 집중하여 학습함으로써 발생할 수 있는 과대적합(Overfitting)을 방지
* 뉴럴 네트워크 후에, 활성화 함수 전에 많이 사용
* p : 각 element들이 0이 될 확률 => 얼마나 끊어줄 것인가

In [13]:
# Dropout 추가
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(p=0.5) # p : probability of an element to be zeroed. Default: 0.5
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.dropout(out) # dropout 추가!
        out = self.relu(out)
        out = self.mlp2(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)

# Batch Normalization

* 배치 정규화는 평균과 분산을 조정하는 과정이 별도의 과정으로 떼어진 것이 아니라, 신경망 안에 포함되어 학습 시 평균과 분산을 조정하는 과정
* 즉, 각 레이어마다 정규화 하는 레이어를 두어, 변형된 분포가 나오지 않도록 조절


In [14]:
# Batch Normalization 추가
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.mlp1 = nn.Linear(input_size, hidden_size)
        self.bn = nn.BatchNorm1d(hidden_size) # batch normalization 1d*(1차원) : BatchNorm1d
        self.relu = nn.ReLU()
        self.mlp2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.mlp1(x)
        out = self.dropout(out) # batch normalization 추가!
        out = self.relu(out)
        out = self.mlp2(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(input_size=28*28*1, hidden_size=100, num_classes=10).to(device)